In [1]:
import pandas as pd
import numpy as np
import shapely
from shapely.geometry import shape, mapping
import geopandas as gpd
from geopandas.tools import sjoin
import requests 
import json
import matplotlib.pyplot as plt

/Users/gil/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
print('Necessary libraries have been imported successfully.')

Necessary libraries have been imported successfully.


In [3]:
tracts2016 = pd.read_csv('plzwrk141619/2016alltractdata.csv')
tracts2019 = pd.read_csv('plzwrk141619/2019alltractdata.csv')

In [4]:
tracts2016.columns

Index(['id', 'Geographic Area Name', 'Estimate!!SEX AND AGE!!Total population',
       'Estimate!!SEX AND AGE!!18 years and over',
       'Percent!!SEX AND AGE!!18 years and over', 'Latino Total Population',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican',
       'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican',
       'Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!

In [5]:
proct16 = tracts2016[['Geographic Area Name','Estimate!!SEX AND AGE!!Total population','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino']]

In [6]:
 tracts2019.columns

Index(['id', 'Geographic Area Name', 'Estimate!!SEX AND AGE!!Total population',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population',
       'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population',
       'Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican',
       'Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!

In [7]:
proct19 = tracts2019[['Geographic Area Name','Estimate!!SEX AND AGE!!Total population','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino']]

In [8]:
proct19.columns

Index(['Geographic Area Name', 'Estimate!!SEX AND AGE!!Total population',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban',
       'Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino'],
      dtype='object')

In [9]:
proct16 = proct16.rename(columns = {'Geographic Area Name':'Tract','Estimate!!SEX AND AGE!!Total population':'Total Population 2016','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'LatinX Population 2016','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican':'Mexican 2016','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican':'Puerto Rican 2016','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban':'Cuban 2016','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino':'Other LatinX 2016'})
proct19 = proct19.rename(columns = {'Geographic Area Name':'Tract','Estimate!!SEX AND AGE!!Total population':'Total Population 2019','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)':'LatinX Population 2019','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Mexican':'Mexican 2019','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican':'Puerto Rican 2019','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Cuban':'Cuban 2019','Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Other Hispanic or Latino':'Other LatinX 2019'})

In [10]:
proct16['Tract'] = proct16['Tract'].astype(str)
proct19['Tract'] = proct19['Tract'].astype(str)

In [11]:
proct16

,Tract,Total Population 2016,LatinX Population 2016,Mexican 2016,Puerto Rican 2016,Cuban 2016,Other LatinX 2016
0,101.0,2962,164,0,105,0,59
1,102.06,3168,10,1,1,0,8
2,102.08,1589,30,17,0,0,13
3,103.04,2107,23,7,0,6,10
4,103.06,2817,53,0,0,0,53
...,...,...,...,...,...,...,...
1473,7601.0,3386,0,0,0,0,0
1474,7611.0,5178,42,0,41,0,1
1475,7612.0,5541,397,101,95,0,201
1476,7613.0,3397,515,9,471,11,24


In [12]:
type(proct19)

pandas.core.frame.DataFrame

In [13]:
merged = pd.merge(proct16,proct19, how = "left", left_on = ['Tract'],right_on =['Tract'])

In [14]:
merged

,Tract,Total Population 2016,LatinX Population 2016,Mexican 2016,Puerto Rican 2016,Cuban 2016,Other LatinX 2016,Total Population 2019,LatinX Population 2019,Mexican 2019,Puerto Rican 2019,Cuban 2019,Other LatinX 2019
0,101.0,2962,164,0,105,0,59,2973,124,6,23,32,63
1,102.06,3168,10,1,1,0,8,3617,44,0,0,26,18
2,102.08,1589,30,17,0,0,13,1122,17,17,0,0,0
3,103.04,2107,23,7,0,6,10,2394,20,0,0,14,6
4,103.06,2817,53,0,0,0,53,2507,23,0,0,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,7601.0,3386,0,0,0,0,0,3441,1,0,0,0,1
1498,7611.0,5178,42,0,41,0,1,5213,62,0,0,0,62
1499,7612.0,5541,397,101,95,0,201,5506,483,154,20,0,309
1500,7613.0,3397,515,9,471,11,24,3259,341,0,161,50,130


In [15]:
permerged = merged.copy()

In [16]:
permerged['% Change in Total Pop.'] = (permerged['Total Population 2019']-permerged['Total Population 2016'])/permerged['Total Population 2016']

In [17]:
permerged['% Change in LatinX Pop.'] = (permerged['LatinX Population 2019']-permerged['LatinX Population 2016'])/permerged['LatinX Population 2016']

In [18]:
permerged['% Change in Mexican Pop.'] = (permerged['Mexican 2019']-permerged['Mexican 2016'])/permerged['Mexican 2016']

In [19]:
permerged['% Change in Puerto Rican Pop.'] = (permerged['Puerto Rican 2019']-permerged['Puerto Rican 2016'])/permerged['Puerto Rican 2016']

In [20]:
permerged['% Change in Cuban Pop.'] = (permerged['Cuban 2019']-permerged['Cuban 2016'])/permerged['Cuban 2016']

In [21]:
permerged['% Change in Other LatinX Pop.'] = (permerged['Other LatinX 2019']-permerged['Other LatinX 2016'])/permerged['Other LatinX 2016']

In [22]:
permerged = permerged[['Tract','% Change in LatinX Pop.','% Change in Mexican Pop.','% Change in Puerto Rican Pop.','% Change in Cuban Pop.','% Change in Other LatinX Pop.','% Change in Total Pop.']]

In [23]:
permerged

,Tract,% Change in LatinX Pop.,% Change in Mexican Pop.,% Change in Puerto Rican Pop.,% Change in Cuban Pop.,% Change in Other LatinX Pop.,% Change in Total Pop.
0,101.0,-0.243902,inf,-0.780952,inf,0.067797,0.003714
1,102.06,3.400000,-1.000000,-1.000000,inf,1.250000,0.141730
2,102.08,-0.433333,0.000000,NaN,NaN,-1.000000,-0.293896
3,103.04,-0.130435,-1.000000,NaN,1.333333,-0.400000,0.136213
4,103.06,-0.566038,NaN,NaN,NaN,-0.566038,-0.110046
...,...,...,...,...,...,...,...
1497,7601.0,inf,NaN,NaN,NaN,inf,0.016243
1498,7611.0,0.476190,NaN,-1.000000,NaN,61.000000,0.006759
1499,7612.0,0.216625,0.524752,-0.789474,NaN,0.537313,-0.006317
1500,7613.0,-0.337864,-1.000000,-0.658174,3.545455,4.416667,-0.040624


In [24]:
permerged['Latitude'] = -40.266666
permerged['Longitude'] = 72.3452

In [25]:
from geopy.geocoders import Nominatim

In [26]:
permerged = gpd.GeoDataFrame(
    permerged, geometry=gpd.points_from_xy(permerged.Longitude, permerged.Latitude))

In [27]:
type(permerged)

geopandas.geodataframe.GeoDataFrame

In [28]:
MA_t = gpd.read_file("CENSUS2010_BLK_BG_TRCT_SHP/CENSUS2010TRACTS_POLY.shp")

In [29]:
MA_t

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,ALAND10,AWATER10,INTPTLAT10,...,AREA_ACRES,POP100_RE,HU100_RE,LOGPL94171,LOGSF1,LOGACS0610,LOGSF1C,SHAPE_AREA,SHAPE_LEN,geometry
0,25,021,418003,25021418003,4180.03,Census Tract 4180.03,G5020,1705668.0,2936.0,+42.2350240,...,422.1740,2481,1350,0141872,123764,0003690,0123764,1.708484e+06,6274.185034,"POLYGON ((240678.759 886748.073, 240607.995 88..."
1,25,021,417701,25021417701,4177.01,Census Tract 4177.01,G5020,1543651.0,12275.0,+42.2523398,...,384.4502,5417,2983,0141838,123730,0003683,0123730,1.555821e+06,6229.137913,"POLYGON ((240969.666 890122.806, 240985.156 89..."
2,25,021,417702,25021417702,4177.02,Census Tract 4177.02,G5020,1685529.0,317947.0,+42.2582818,...,416.4918,2765,1109,0141843,123735,0003684,0123735,1.685489e+06,8542.498286,"POLYGON ((242328.221 890545.050, 242276.888 89..."
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,191.1397,3377,1676,0141882,123774,0003693,0123774,7.735182e+05,4540.296101,"POLYGON ((240372.666 888331.030, 240393.057 88..."
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,325.2812,4280,1987,0141874,123766,0003691,0123766,1.316372e+06,5854.497366,"POLYGON ((240687.299 888298.408, 240700.449 88..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,25,027,709501,25027709501,7095.01,Census Tract 7095.01,G5020,5484847.0,0.0,+42.5343643,...,1355.2791,2149,809,0189917,171462,0004065,0171462,5.484642e+06,9858.734830,"POLYGON ((182231.394 921599.665, 182405.418 92..."
1468,25,027,709502,25027709502,7095.02,Census Tract 7095.02,G5020,6089034.0,1287.0,+42.5573788,...,1504.8969,5670,2488,0189919,171464,0004066,0171464,6.090126e+06,11906.751110,"POLYGON ((181045.532 923444.745, 181902.923 92..."
1469,25,027,730801,25027730801,7308.01,Census Tract 7308.01,G5020,3158638.0,7980.0,+42.2816214,...,782.4305,3665,1557,0190133,171678,0004114,0171678,3.166396e+06,9559.778037,"POLYGON ((174482.678 891951.535, 174407.978 89..."
1470,25,027,710800,25027710800,7108,Census Tract 7108,G5020,2337125.0,0.0,+42.5913348,...,577.4996,4539,1940,0189967,171512,0004077,0171512,2.337067e+06,9574.497510,"POLYGON ((176303.717 926403.339, 176284.512 92..."


In [30]:
Tractjoin = gpd.pd.merge(MA_t, permerged,  how='left', left_on=['NAME10'], right_on = ['Tract'])

In [33]:
Tractjoin

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,ALAND10,AWATER10,INTPTLAT10,...,Tract,% Change in LatinX Pop.,% Change in Mexican Pop.,% Change in Puerto Rican Pop.,% Change in Cuban Pop.,% Change in Other LatinX Pop.,% Change in Total Pop.,Latitude,Longitude,geometry_y
0,25,021,418003,25021418003,4180.03,Census Tract 4180.03,G5020,1705668.0,2936.0,+42.2350240,...,4180.03,-0.458824,-0.612245,inf,NaN,-0.479339,0.016760,-40.266666,72.3452,POINT (72.34520 -40.26667)
1,25,021,417701,25021417701,4177.01,Census Tract 4177.01,G5020,1543651.0,12275.0,+42.2523398,...,4177.01,0.339744,0.704918,NaN,0.48,-0.311111,0.033125,-40.266666,72.3452,POINT (72.34520 -40.26667)
2,25,021,417702,25021417702,4177.02,Census Tract 4177.02,G5020,1685529.0,317947.0,+42.2582818,...,4177.02,-0.095238,-1.000000,0.285714,-1.00,0.846154,-0.030563,-40.266666,72.3452,POINT (72.34520 -40.26667)
3,25,021,418102,25021418102,4181.02,Census Tract 4181.02,G5020,771203.0,2371.0,+42.2470410,...,4181.02,-0.671429,-1.000000,-0.904110,NaN,-0.370968,-0.098172,-40.266666,72.3452,POINT (72.34520 -40.26667)
4,25,021,418004,25021418004,4180.04,Census Tract 4180.04,G5020,1316466.0,0.0,+42.2383744,...,4180.04,0.523810,inf,0.062500,NaN,0.200000,0.061053,-40.266666,72.3452,POINT (72.34520 -40.26667)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,25,027,709501,25027709501,7095.01,Census Tract 7095.01,G5020,5484847.0,0.0,+42.5343643,...,7095.01,0.445783,-0.192308,3.212121,NaN,-0.252336,-0.027548,-40.266666,72.3452,POINT (72.34520 -40.26667)
1468,25,027,709502,25027709502,7095.02,Census Tract 7095.02,G5020,6089034.0,1287.0,+42.5573788,...,7095.02,0.076677,inf,0.150725,inf,-0.626335,0.020983,-40.266666,72.3452,POINT (72.34520 -40.26667)
1469,25,027,730801,25027730801,7308.01,Census Tract 7308.01,G5020,3158638.0,7980.0,+42.2816214,...,7308.01,0.204724,2.823529,1.103448,NaN,-0.666667,0.022868,-40.266666,72.3452,POINT (72.34520 -40.26667)
1470,25,027,710800,25027710800,7108,Census Tract 7108,G5020,2337125.0,0.0,+42.5913348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [34]:
Tractjoin = Tractjoin.drop(columns = ["Latitude","Longitude"])

In [35]:
MA_p = gpd.read_file("wardsprecincts_poly/WARDSPRECINCTS_POLY.shp")
MA_p['WARD'] = MA_p['WARD'].astype(str)

In [36]:
pres2016 = pd.read_csv('2016PresidentPrecinct.csv')
pres2020 = pd.read_csv('2020PresidentPrecint.csv')

In [37]:
pres2020.dtypes

City/Town            object
Ward                 object
Precinct             object
Democratic          float64
Republican          float64
 Libertarian         object
Green-Rainbow       float64
All Others          float64
No Preference       float64
Blanks              float64
Total Votes Cast    float64
dtype: object

In [38]:
pres2020

,City/Town,Ward,Precinct,Democratic,Republican,Libertarian,Green-Rainbow,All Others,No Preference,Blanks,Total Votes Cast
0,Abington,-,1,1037.0,764.0,29.00,6.0,8.0,0.0,16.0,1860.0
1,Abington,-,2,960.0,838.0,23.00,16.0,4.0,0.0,9.0,1850.0
2,Abington,-,3,954.0,848.0,31.00,8.0,3.0,0.0,9.0,1853.0
3,Abington,-,4,1117.0,904.0,28.00,7.0,6.0,0.0,26.0,2088.0
4,Abington,-,5,1141.0,882.0,35.00,8.0,3.0,0.0,15.0,2084.0
...,...,...,...,...,...,...,...,...,...,...,...
2099,Yarmouth,-,3,1145.0,672.0,29.00,13.0,9.0,0.0,8.0,1876.0
2100,Yarmouth,-,4,1220.0,867.0,22.00,19.0,6.0,0.0,14.0,2148.0
2101,Yarmouth,-,5,1100.0,848.0,28.00,8.0,1.0,0.0,19.0,2004.0
2102,Yarmouth,-,6,1145.0,758.0,16.00,11.0,11.0,0.0,18.0,1959.0


In [39]:
pres2016 = pres2016.rename(columns = {'Clinton/ Kaine':'Democratic 2016','Trump/ Pence':'Republican 2016'})
pres2016 = pres2016[['City/Town','Pct','Ward','Democratic 2016','Republican 2016','Total Votes Cast']]
pres2020 = pres2020.rename(columns = {'Democratic': 'Democratic 2020','Republican':'Republican 2020','Precinct':'Pct','Total Votes Cast':'Total Votes Cast 2020'})
pres2020 = pres2020[['City/Town','Pct','Ward','Democratic 2020','Republican 2020','Total Votes Cast 2020']]

#pres2016.loc[pres2016['Ward']=='-','Ward'] = 'None'
#pres2020.loc[pres2020['Ward']=='None','Ward'] = '-'
pres2016['City/Town']=pres2016['City/Town'].str.upper()
pres2020['City/Town']=pres2020['City/Town'].str.upper()

In [40]:
pres2016

,City/Town,Pct,Ward,Democratic 2016,Republican 2016,Total Votes Cast
0,Abington,1,-,818.0,717.0,1709.0
1,Abington,2,-,739.0,785.0,1671.0
2,Abington,3,-,773.0,808.0,1726.0
3,Abington,4,-,877.0,878.0,1919.0
4,Abington,5,-,908.0,829.0,1887.0
...,...,...,...,...,...,...
2169,Yarmouth,3,-,904.0,663.0,1679.0
2170,Yarmouth,4,-,1002.0,860.0,2038.0
2171,Yarmouth,5,-,881.0,797.0,1805.0
2172,Yarmouth,6,-,907.0,753.0,1804.0


In [41]:
#pres2016 = pres2016.drop(2175)

In [42]:
#pres2020=pres2020.drop(2318)

In [43]:
pres2020 = pres2020.dropna()
pres2016 = pres2016.dropna()

In [44]:
pres2016

,City/Town,Pct,Ward,Democratic 2016,Republican 2016,Total Votes Cast
0,Abington,1,-,818.0,717.0,1709.0
1,Abington,2,-,739.0,785.0,1671.0
2,Abington,3,-,773.0,808.0,1726.0
3,Abington,4,-,877.0,878.0,1919.0
4,Abington,5,-,908.0,829.0,1887.0
...,...,...,...,...,...,...
2169,Yarmouth,3,-,904.0,663.0,1679.0
2170,Yarmouth,4,-,1002.0,860.0,2038.0
2171,Yarmouth,5,-,881.0,797.0,1805.0
2172,Yarmouth,6,-,907.0,753.0,1804.0


pres2016['Democratic 2016'] = pres2016['Democratic 2016'].str.replace(',','').astype('float')
pres2016['Republican 2016'] = pres2016['Republican 2016'].str.replace(',','').astype('float')
pres2016['Total Votes Cast'] = pres2016['Total Votes Cast'].str.replace(',','').astype('float')

In [45]:
# pres2016['Ward'] = pres2016['Ward'].astype(str)
# pres2020['Ward'] = pres2020['Ward'].astype(str)
# pres2016['Pct'] = pres2016['Pct'].astype(str)
# pres2016['City/Town'] = pres2016['City/Town'].astype(str)
# pres2020['Pct'] = pres2020['Pct'].astype(str)
# pres2020['City/Town'] = pres2020['City/Town'].astype(str)

In [46]:
#pres2016['Ward'].replace("None", '-', inplace=True)

In [47]:
pres2020.dtypes

City/Town                 object
Pct                       object
Ward                      object
Democratic 2020          float64
Republican 2020          float64
Total Votes Cast 2020    float64
dtype: object

In [48]:
pres2016.dtypes

City/Town            object
Pct                  object
Ward                 object
Democratic 2016     float64
Republican 2016     float64
Total Votes Cast    float64
dtype: object

In [49]:
pres2016['Ward'] = pres2016['Ward'].str.replace('-','0')
pres2020['Ward'] = pres2020['Ward'].str.replace('-','0')

In [50]:
pres2016

,City/Town,Pct,Ward,Democratic 2016,Republican 2016,Total Votes Cast
0,Abington,1,0,818.0,717.0,1709.0
1,Abington,2,0,739.0,785.0,1671.0
2,Abington,3,0,773.0,808.0,1726.0
3,Abington,4,0,877.0,878.0,1919.0
4,Abington,5,0,908.0,829.0,1887.0
...,...,...,...,...,...,...
2169,Yarmouth,3,0,904.0,663.0,1679.0
2170,Yarmouth,4,0,1002.0,860.0,2038.0
2171,Yarmouth,5,0,881.0,797.0,1805.0
2172,Yarmouth,6,0,907.0,753.0,1804.0


In [51]:
pres2020[90:100]

,City/Town,Pct,Ward,Democratic 2020,Republican 2020,Total Votes Cast 2020
90,Attleboro,A,3,1276.0,753.0,2089.0
91,Attleboro,B,3,1242.0,758.0,2056.0
92,Attleboro,A,4,1038.0,530.0,1628.0
93,Attleboro,B,4,1452.0,1113.0,2620.0
94,Attleboro,A,5,801.0,382.0,1242.0
95,Attleboro,B,5,1144.0,749.0,1951.0
96,Attleboro,A,6,1036.0,795.0,1900.0
97,Attleboro,B,6,1170.0,831.0,2087.0
98,Auburn,1,0,1172.0,916.0,2171.0
99,Auburn,2,0,1160.0,910.0,2123.0


In [52]:
mergedP = pd.merge(pres2016,pres2020, how = 'inner', left_on = ['City/Town','Pct','Ward'],right_on = ['City/Town','Pct','Ward'])

In [53]:
mergedP

,City/Town,Pct,Ward,Democratic 2016,Republican 2016,Total Votes Cast,Democratic 2020,Republican 2020,Total Votes Cast 2020
0,Attleboro,A,1,761.0,761.0,1638.0,1006.0,800.0,1861.0
1,Attleboro,B,1,779.0,758.0,1666.0,1071.0,806.0,1925.0
2,Attleboro,A,2,894.0,774.0,1854.0,1157.0,813.0,2037.0
3,Attleboro,B,2,949.0,861.0,1988.0,1268.0,942.0,2295.0
4,Attleboro,A,3,933.0,650.0,1759.0,1276.0,753.0,2089.0
...,...,...,...,...,...,...,...,...,...
970,Worcester,1,10,651.0,180.0,896.0,675.0,244.0,944.0
971,Worcester,2,10,676.0,124.0,875.0,777.0,175.0,992.0
972,Worcester,3,10,777.0,89.0,926.0,645.0,194.0,877.0
973,Worcester,4,10,612.0,101.0,753.0,525.0,214.0,763.0


In [54]:
#SOLVE NON WARD PRECINCTS

In [55]:
mergedP['Latitude'] = -40.266666
mergedP['Longitude'] = 72.3452

In [56]:
mergedPG = gpd.GeoDataFrame(
   mergedP, geometry=gpd.points_from_xy(mergedP.Longitude, mergedP.Latitude))

In [57]:
mergedPG = mergedPG.drop(columns = ["Latitude","Longitude"])

In [58]:
mergedPG

,City/Town,Pct,Ward,Democratic 2016,Republican 2016,Total Votes Cast,Democratic 2020,Republican 2020,Total Votes Cast 2020,geometry
0,Attleboro,A,1,761.0,761.0,1638.0,1006.0,800.0,1861.0,POINT (72.34520 -40.26667)
1,Attleboro,B,1,779.0,758.0,1666.0,1071.0,806.0,1925.0,POINT (72.34520 -40.26667)
2,Attleboro,A,2,894.0,774.0,1854.0,1157.0,813.0,2037.0,POINT (72.34520 -40.26667)
3,Attleboro,B,2,949.0,861.0,1988.0,1268.0,942.0,2295.0,POINT (72.34520 -40.26667)
4,Attleboro,A,3,933.0,650.0,1759.0,1276.0,753.0,2089.0,POINT (72.34520 -40.26667)
...,...,...,...,...,...,...,...,...,...,...
970,Worcester,1,10,651.0,180.0,896.0,675.0,244.0,944.0,POINT (72.34520 -40.26667)
971,Worcester,2,10,676.0,124.0,875.0,777.0,175.0,992.0,POINT (72.34520 -40.26667)
972,Worcester,3,10,777.0,89.0,926.0,645.0,194.0,877.0,POINT (72.34520 -40.26667)
973,Worcester,4,10,612.0,101.0,753.0,525.0,214.0,763.0,POINT (72.34520 -40.26667)


In [59]:
mergedPG['City/Town']=mergedPG['City/Town'].str.upper()

In [60]:
mergedPG['% Change in Democratic Votes'] = (mergedPG['Democratic 2020']-mergedPG['Democratic 2016'])/mergedPG['Democratic 2016']

In [61]:
mergedPG['% Change in Total Votes'] = (mergedPG['Total Votes Cast 2020']-mergedPG['Total Votes Cast'])/mergedPG['Total Votes Cast']

In [62]:
mergedPG['% Change in Republican Votes'] = (mergedPG['Republican 2020']-mergedPG['Republican 2016'])/mergedPG['Republican 2016']

In [63]:
finalP = mergedPG[['City/Town','Pct','Ward','% Change in Democratic Votes','% Change in Republican Votes','% Change in Total Votes','geometry']]

In [64]:
type(finalP)

geopandas.geodataframe.GeoDataFrame

In [65]:
finalP

,City/Town,Pct,Ward,% Change in Democratic Votes,% Change in Republican Votes,% Change in Total Votes,geometry
0,ATTLEBORO,A,1,0.321945,0.051248,0.136142,POINT (72.34520 -40.26667)
1,ATTLEBORO,B,1,0.374840,0.063325,0.155462,POINT (72.34520 -40.26667)
2,ATTLEBORO,A,2,0.294183,0.050388,0.098706,POINT (72.34520 -40.26667)
3,ATTLEBORO,B,2,0.336143,0.094077,0.154427,POINT (72.34520 -40.26667)
4,ATTLEBORO,A,3,0.367631,0.158462,0.187607,POINT (72.34520 -40.26667)
...,...,...,...,...,...,...,...
970,WORCESTER,1,10,0.036866,0.355556,0.053571,POINT (72.34520 -40.26667)
971,WORCESTER,2,10,0.149408,0.411290,0.133714,POINT (72.34520 -40.26667)
972,WORCESTER,3,10,-0.169884,1.179775,-0.052916,POINT (72.34520 -40.26667)
973,WORCESTER,4,10,-0.142157,1.118812,0.013280,POINT (72.34520 -40.26667)


In [66]:
MA_p = gpd.read_file("wardsprecincts_poly/WARDSPRECINCTS_POLY.shp")

In [67]:
MA_p['WARD'] = MA_p['WARD'].astype(str)

In [68]:
MA_p.head()

,WP_NAME,WARD,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,geometry
0,Braintree Town Precinct 5B,None,5B,5B,2883,BRAINTREE,40,1.217,778.80,2012,3.151713e+06,8711.937896,"POLYGON ((242364.517 883741.979, 242359.642 88..."
1,Braintree Town Precinct 6A,None,6A,6A,3070,BRAINTREE,40,1.994,1275.96,2012,5.163667e+06,11175.567690,"POLYGON ((243479.798 881994.897, 243477.538 88..."
2,Braintree Town Precinct 6B,None,6B,6B,2925,BRAINTREE,40,1.134,725.97,2012,2.937925e+06,9455.882837,"POLYGON ((241614.364 881551.551, 241602.936 88..."
3,Chelsea City Ward 1 Precinct 1,1,1,1-1,2295,CHELSEA,57,0.139,88.82,2012,3.594632e+05,3340.823153,"POLYGON ((239281.036 904133.287, 239277.513 90..."
4,Chelsea City Ward 1 Precinct 2,1,2,1-2,2108,CHELSEA,57,0.073,47.04,2012,1.903509e+05,2247.883591,"POLYGON ((238408.578 904480.138, 238393.140 90..."


In [69]:
joinP = gpd.pd.merge(MA_p, finalP,  how='left', left_on=['PRECINCT','WARD','TOWN'], right_on = ['Pct','Ward','City/Town'])

In [70]:
joinP

,WP_NAME,WARD,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,geometry_x,City/Town,Pct,Ward,% Change in Democratic Votes,% Change in Republican Votes,% Change in Total Votes,geometry_y
0,Braintree Town Precinct 5B,None,5B,5B,2883,BRAINTREE,40,1.217000,778.800000,2012,3.151713e+06,8711.937896,"POLYGON ((242364.517 883741.979, 242359.642 88...",NaN,NaN,NaN,NaN,NaN,NaN,None
1,Braintree Town Precinct 6A,None,6A,6A,3070,BRAINTREE,40,1.994000,1275.960000,2012,5.163667e+06,11175.567690,"POLYGON ((243479.798 881994.897, 243477.538 88...",NaN,NaN,NaN,NaN,NaN,NaN,None
2,Braintree Town Precinct 6B,None,6B,6B,2925,BRAINTREE,40,1.134000,725.970000,2012,2.937925e+06,9455.882837,"POLYGON ((241614.364 881551.551, 241602.936 88...",NaN,NaN,NaN,NaN,NaN,NaN,None
3,Chelsea City Ward 1 Precinct 1,1,1,1-1,2295,CHELSEA,57,0.139000,88.820000,2012,3.594632e+05,3340.823153,"POLYGON ((239281.036 904133.287, 239277.513 90...",CHELSEA,1,1,-0.013986,0.630435,0.029668,POINT (72.34520 -40.26667)
4,Chelsea City Ward 1 Precinct 2,1,2,1-2,2108,CHELSEA,57,0.073000,47.040000,2012,1.903509e+05,2247.883591,"POLYGON ((238408.578 904480.138, 238393.140 90...",CHELSEA,2,1,-0.043771,0.954545,0.005319,POINT (72.34520 -40.26667)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,Boston City Ward 7 Precinct 8,7,8,7-8,1713,BOSTON,35,0.251525,160.975880,2016,6.514463e+05,4262.156864,"POLYGON ((236328.257 897806.825, 236339.793 89...",BOSTON,8,7,0.289557,0.365269,0.245655,POINT (72.34520 -40.26667)
2148,Boston City Ward 12 Precinct 3,12,3,12-3,1082,BOSTON,35,0.077683,49.716987,2016,2.011975e+05,2455.103261,"POLYGON ((234387.165 896770.015, 234357.873 89...",BOSTON,3,12,-0.116861,0.833333,-0.138088,POINT (72.34520 -40.26667)
2149,Boston City Ward 12 Precinct 2,12,2,12-2,3282,BOSTON,35,0.143595,91.900693,2016,3.719089e+05,3754.545614,"POLYGON ((234109.165 896499.623, 234101.151 89...",BOSTON,2,12,0.141208,1.961538,0.146986,POINT (72.34520 -40.26667)
2150,Boston City Ward 11 Precinct 2,11,2,11-2,2172,BOSTON,35,0.120155,76.899177,2016,3.111999e+05,3381.654371,"POLYGON ((233908.714 896981.859, 233743.883 89...",BOSTON,2,11,0.021000,1.360000,0.019571,POINT (72.34520 -40.26667)


In [71]:
joinP=joinP.drop(columns = 'geometry_y')
joinP=joinP.rename(columns = {'geometry_x':'geometry'})

In [72]:
joinP.columns

Index(['WP_NAME', 'WARD', 'PRECINCT', 'DISTRICT', 'POP_2010', 'TOWN',
       'TOWN_ID', 'AREA_SQMI', 'AREA_ACRES', 'YEAR', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry', 'City/Town', 'Pct', 'Ward', '% Change in Democratic Votes',
       '% Change in Republican Votes', '% Change in Total Votes'],
      dtype='object')

In [73]:
joinP

,WP_NAME,WARD,PRECINCT,DISTRICT,POP_2010,TOWN,TOWN_ID,AREA_SQMI,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,geometry,City/Town,Pct,Ward,% Change in Democratic Votes,% Change in Republican Votes,% Change in Total Votes
0,Braintree Town Precinct 5B,None,5B,5B,2883,BRAINTREE,40,1.217000,778.800000,2012,3.151713e+06,8711.937896,"POLYGON ((242364.517 883741.979, 242359.642 88...",NaN,NaN,NaN,NaN,NaN,NaN
1,Braintree Town Precinct 6A,None,6A,6A,3070,BRAINTREE,40,1.994000,1275.960000,2012,5.163667e+06,11175.567690,"POLYGON ((243479.798 881994.897, 243477.538 88...",NaN,NaN,NaN,NaN,NaN,NaN
2,Braintree Town Precinct 6B,None,6B,6B,2925,BRAINTREE,40,1.134000,725.970000,2012,2.937925e+06,9455.882837,"POLYGON ((241614.364 881551.551, 241602.936 88...",NaN,NaN,NaN,NaN,NaN,NaN
3,Chelsea City Ward 1 Precinct 1,1,1,1-1,2295,CHELSEA,57,0.139000,88.820000,2012,3.594632e+05,3340.823153,"POLYGON ((239281.036 904133.287, 239277.513 90...",CHELSEA,1,1,-0.013986,0.630435,0.029668
4,Chelsea City Ward 1 Precinct 2,1,2,1-2,2108,CHELSEA,57,0.073000,47.040000,2012,1.903509e+05,2247.883591,"POLYGON ((238408.578 904480.138, 238393.140 90...",CHELSEA,2,1,-0.043771,0.954545,0.005319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,Boston City Ward 7 Precinct 8,7,8,7-8,1713,BOSTON,35,0.251525,160.975880,2016,6.514463e+05,4262.156864,"POLYGON ((236328.257 897806.825, 236339.793 89...",BOSTON,8,7,0.289557,0.365269,0.245655
2148,Boston City Ward 12 Precinct 3,12,3,12-3,1082,BOSTON,35,0.077683,49.716987,2016,2.011975e+05,2455.103261,"POLYGON ((234387.165 896770.015, 234357.873 89...",BOSTON,3,12,-0.116861,0.833333,-0.138088
2149,Boston City Ward 12 Precinct 2,12,2,12-2,3282,BOSTON,35,0.143595,91.900693,2016,3.719089e+05,3754.545614,"POLYGON ((234109.165 896499.623, 234101.151 89...",BOSTON,2,12,0.141208,1.961538,0.146986
2150,Boston City Ward 11 Precinct 2,11,2,11-2,2172,BOSTON,35,0.120155,76.899177,2016,3.111999e+05,3381.654371,"POLYGON ((233908.714 896981.859, 233743.883 89...",BOSTON,2,11,0.021000,1.360000,0.019571


In [74]:
FinalData = gpd.sjoin(finalP, joinP, how="left", op="intersects")

<ipython-input-74-bc363ec5c21a>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:26986

  FinalData = gpd.sjoin(finalP, joinP, how="left", op="intersects")


In [75]:
FinalData

,City/Town_left,Pct_left,Ward_left,% Change in Democratic Votes_left,% Change in Republican Votes_left,% Change in Total Votes_left,geometry,index_right,WP_NAME,WARD,...,AREA_ACRES,YEAR,SHAPE_AREA,SHAPE_LEN,City/Town_right,Pct_right,Ward_right,% Change in Democratic Votes_right,% Change in Republican Votes_right,% Change in Total Votes_right
0,ATTLEBORO,A,1,0.321945,0.051248,0.136142,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATTLEBORO,B,1,0.374840,0.063325,0.155462,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATTLEBORO,A,2,0.294183,0.050388,0.098706,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ATTLEBORO,B,2,0.336143,0.094077,0.154427,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ATTLEBORO,A,3,0.367631,0.158462,0.187607,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,WORCESTER,1,10,0.036866,0.355556,0.053571,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
971,WORCESTER,2,10,0.149408,0.411290,0.133714,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
972,WORCESTER,3,10,-0.169884,1.179775,-0.052916,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
973,WORCESTER,4,10,-0.142157,1.118812,0.013280,POINT (72.34520 -40.26667),NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
